In [40]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import pickle
from flask import Flask, request, jsonify
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.base import BaseEstimator
from sklearn.preprocessing import MinMaxScaler

In [22]:
# Load the dataset
df = pd.read_csv('combined_data.csv')

In [32]:
# Extract relevant features
features = ['temperature', 'wind_speed', 'weather_main', 'datequery', 'dayquery', 'hourquery',
            'pressure', 'humidity', 'available_bikes', 'name']
df_features = df[features]

In [33]:
# Prepare the data
# One-hot encode the non-numerical data
encoded_data = pd.get_dummies(
    df_features, columns=['weather_main', 'name', 'datequery', 'dayquery', 'hourquery'])

In [37]:
# Split the data using TimeSeriesSplit
cv = TimeSeriesSplit(n_splits=5)
for train_index, test_index in cv.split(df_features):
    train_data = df_features.iloc[train_index]
    test_data = df_features.iloc[test_index]

# Maybe use this option for data split
"""spilts = []
for train_index, test_index in cv.split(df_features):
    splits.append((train_index, test_index))"""

In [36]:
# Define the parameter grid to search
p_values = [0, 1, 2, 3]
d_values = [0, 1]
q_values = [0, 1, 2, 3]
parameters = {'p': p_values, 'd': d_values, 'q': q_values}

In [43]:
class ARIMAEstimator(BaseEstimator):
    def __init__(self, p, d, q):
        self.p = p
        self.d = d
        self.q = q

    def fit(self, X, y):
        self.model = ARIMA(y, order=(self.p, self.d, self.q))
        self.results = self.model.fit()

    def predict(self, X):
        return self.results.forecast(len(X))
    
    def score(self, X, y):
        y_pred = self.predict(X)
        mse = ((y_pred - y) ** 2).mean()
        return -mse

In [45]:
grid_search = GridSearchCV(model, parameters, scoring='neg_mean_squared_error')
grid_search.fit(train_data.drop('available_bikes', axis=1),
                train_data['available_bikes'])

"""# Train the time-series forecasting model
model = ARIMAEstimator(p=1, d=0, q=1)

# Perform Grid Search to find the optimal order of the model
grid_search = GridSearchCV(model, parameters)
grid_search.fit(train_data.drop('available_bikes', axis=1),
                train_data['available_bikes'])"""

c:\Users\Olga\anaconda3\envs\comp30830\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Olga\anaconda3\envs\comp30830\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Olga\anaconda3\envs\comp30830\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Olga\anaconda3\envs\comp30830\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\Olga\anaconda3\envs\comp30830\lib\site-packages\statsmodels\tsa\base\tsa_model.py:4

KeyboardInterrupt: 

In [ ]:
# Get the optimal order of the model
optimal_order = grid_search.best_params_
print('Optimal Order:', optimal_order)

In [ ]:
# Train the ARIMA model with the optimal order
optimal_model = ARIMAEstimator(**optimal_order)
optimal_model.fit(train_data.drop('available_bikes', axis=1),
                  train_data['available_bikes'])
optimal_predictions = optimal_model.predict(
    test_data.drop('available_bikes', axis=1))
mse = ((optimal_predictions - test_data['available_bikes']) ** 2).mean()
print(mse)

In [38]:
# Training without creating a class to make ARIMA be able to use Grid Search
"""# Train the time-series forecasting model
model = ARIMA(train_data['available_bikes'], order=(1, 0, 1))
#results = model.fit()"""

In [39]:
"""# Perform Grid Search to find the optimal order of the model
grid_search = GridSearchCV(model, parameters)
grid_search.fit(train_data['available_bikes'])"""

TypeError: Cannot clone object '<statsmodels.tsa.arima.model.ARIMA object at 0x0000026142FC98E0>' (type <class 'statsmodels.tsa.arima.model.ARIMA'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [30]:
"""# Get the optimal order of the model
optimal_order = grid_search.best_params_
print('Optimal Order:', optimal_order)"""

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [ ]:
"""# Train the ARIMA model with the optimal order
optimal_model = ARIMA(train_data['available_bikes'], order=(
    optimal_order['p'], optimal_order['d'], optimal_order['q']))
optimal_results = optimal_model.fit()"""

In [ ]:
"""# Evaluate the model performance on the test set
predictions = optimal_results.forecast(len(test_data))
mse = ((predictions - test_data['available_bikes']) ** 2).mean()

# Evaluate the model performance on the test set
predictions = results.forecast(len(test_data))
mse = ((predictions - test_data['available_bikes']) ** 2).mean()
print(mse)"""

88.83466870223899


In [ ]:
# Serialize the trained model object and save it to a file
"""with open('model.pkl', 'wb') as f:
    pickle.dump(results, f)"""

with open('model.pkl', 'wb') as handle:
    pickle.dump(model, handle, pickle.HIGHEST_PROTOCOL)


In [ ]:
# Build the Flask API endpoint
app = Flask(__name__)
with open('model.pkl', 'rb') as handle:
    model = pickle.load(handle)

@app.route('/predict')
# Load the trained model object from the file

def predict():
    # Call the predict method over model:
        result = model.predict(test_data)

        # Convert the result to a dictionary and return as a JSON object
        result_dict = {'bike_availability':result.tolist()}
        return jsonify(result_dict)

"""def predict():
    # Deserialize the trained model object
    with open('model.pkl', 'rb') as handle:
        model = pickle.load(handle)

    # Get inputs from the GET request
    weather_data = [float(request.args.get('temperature')),
                    float(request.args.get('wind_speed')),
                    float(request.args.get('pressure')),
                    float(request.args.get('humidity'))]
    date_query = pd.to_datetime(request.args.get('date_query'))

    # Use the model to make predictions
    prediction = model.forecast(steps=1, exog=[weather_data])[0]

    # Return the prediction as a JSON object
    return jsonify({'bike_availability': prediction})"""

"def predict():\n    # Deserialize the trained model object\n    with open('model.pkl', 'rb') as handle:\n        model = pickle.load(handle)\n\n    # Get inputs from the GET request\n    weather_data = [float(request.args.get('temperature')),\n                    float(request.args.get('wind_speed')),\n                    float(request.args.get('pressure')),\n                    float(request.args.get('humidity'))]\n    date_query = pd.to_datetime(request.args.get('date_query'))\n\n    # Use the model to make predictions\n    prediction = model.forecast(steps=1, exog=[weather_data])[0]\n\n    # Return the prediction as a JSON object\n    return jsonify({'bike_availability': prediction})"

In [ ]:
"""# Call the predict() method on the loaded model object
start_idx = len(test_data)
end_idx = start_idx + 10  # predict the next 10 values
output = model.predict(start=start_idx, end=end_idx)

print(output)"""

'# Call the predict() method on the loaded model object\nstart_idx = len(test_data)\nend_idx = start_idx + 10  # predict the next 10 values\noutput = model.predict(start=start_idx, end=end_idx)\n\nprint(output)'

In [ ]:
# Load the trained model object from the file
with open('model.pkl', 'rb') as handle:
    model = pickle.load(handle)

# Call the predict() method on the loaded model object to generate forecasts
output = model.predict(start=train_data.index[0])

# Convert the pandas Series object to a pandas DataFrame
df_output = pd.DataFrame(output, columns=['predicted_mean'])

# Plot the forecasted values against the actual values
plt.plot(train_data.index, train_data.values, label='Actual')
plt.plot(df_output.index, df_output['predicted_mean'], label='Predicted')
plt.legend()
plt.show()

TypeError: predict() missing 1 required positional argument: 'params'

In [ ]:
print(test_data.index)

Int64Index([14220, 14221, 14222, 14223, 14224, 14225, 14226, 14227, 14228,
            14229,
            ...
            17053, 17054, 17055, 17056, 17057, 17058, 17059, 17060, 17061,
            17062],
           dtype='int64', length=2843)


In [ ]:
# Define the MinMaxScaler object
scaler = MinMaxScaler()

# Fit the scaler object on your data
scaler.fit(test_data)

# Load the trained model object from the file
with open('model.pkl', 'rb') as handle:
    model = pickle.load(handle)

# Call the predict() method on the loaded model object to generate forecasts
output = model.predict(start=test_data.index[0])

# Convert the pandas Series object to a pandas DataFrame
df_output = pd.DataFrame(output, columns=['predicted_mean'])

# Inverse transform the predicted values to get them back to their original scale
predicted_values = scaler.inverse_transform(df_output)

# Convert the numpy array to a pandas DataFrame and set the index
df_predicted = pd.DataFrame(
    predicted_values, index=df_output.index, columns=['predicted_mean'])

# Print the forecasted values
print(df_predicted)

ValueError: could not convert string to float: 'Clouds'